In [1]:
import os
os.environ["MANTLE"] = "lattice"
import silica as si
import magma as m

@si.coroutine(inputs={"I": si.Bit})
def TFF(init=False):
    value = init
    O = value
    while True:
        I = yield O
        O = value
        value = I ^ O
        
TFF = si.compile(TFF(), file_name="build/silica_tff.py")
        
with open("build/silica_tff.py", "r") as magma_file:
    print(magma_file.read())

import mantle lattice ice40
import mantle lattice mantle40
from magma import *
import os
os.environ["MANTLE"] = os.getenv("MANTLE", "coreir")
from mantle import *
import mantle.common.operator

TFF = DefineCircuit("TFF", "O", Out(Bit), "I", In(Bit), *ClockInterface(has_ce=False))
CE = VCC
I = TFF.I
value = DFF(has_ce=True, name="value", init=0)
value_CE = [CE]
wireclock(TFF, value)
O_0_tmp = value.O
value_next_0_tmp = xor(I, O_0_tmp)
wire(value_next_0_tmp, value.I)
wire(O_0_tmp, TFF.O)
if len(value_CE) == 1:
    wire(value_CE[0], value.CE)
else:
    wire(and_(*value_CE), value.CE)
EndDefine()


In [2]:
from loam.boards.icestick import IceStick
from mantle import Counter

icestick = IceStick()
icestick.Clock.on()
for i in range(len(icestick.J1)):
    icestick.J1[i].output().on()
main = icestick.main()
tff = TFF()
counter = Counter(2)
m.wire(counter.O[1], tff.I)
m.wire(tff.O, main.J1[0])
m.wire(m.bits(0, 7), main.J1[1:])

m.compile("build/silica_tff", main)

compiling XOr2
compiling TFF
compiling FullAdder
compiling Add2Cout
compiling Register2
compiling Counter2
compiling main


In [3]:
%%bash
yosys -Q -p 'synth_ice40 -top main -blif build/silica_tff.blif' build/silica_tff.v
arachne-pnr -q -d 1k -o build/silica_tff.txt -p build/silica_tff.pcf build/silica_tff.blif
icepack build/silica_tff.txt build/silica_tff.bin
icetime -tmd hx1k build/silica_tff.txt


-- Parsing `build/silica_tff.v' using frontend `verilog' --

1. Executing Verilog-2005 frontend.
Parsing Verilog input from `build/silica_tff.v' to AST representation.
Generating RTLIL representation for module `\XOr2'.
Generating RTLIL representation for module `\TFF'.
Generating RTLIL representation for module `\FullAdder'.
Generating RTLIL representation for module `\Add2Cout'.
Generating RTLIL representation for module `\Register2'.
Generating RTLIL representation for module `\Counter2'.
Generating RTLIL representation for module `\main'.
Successfully finished Verilog frontend.

-- Running command `synth_ice40 -top main -blif build/silica_tff.blif' --

2. Executing SYNTH_ICE40 pass.

2.1. Executing Verilog-2005 frontend.
Parsing Verilog input from `/usr/local/bin/../share/yosys/ice40/cells_sim.v' to AST representation.
Generating RTLIL representation for module `\SB_IO'.
Generating RTLIL representation for module `\SB_GB_IO'.
Generating RTLIL representation for module `\SB_GB'.
Ge

In [4]:
from mantle import DFF

TFF = m.DefineCircuit('TFF', 'I', m.In(m.Bit), 'O', m.Out(m.Bit), 'CLK', m.In(m.Bit))

# create ff holding state first
ff = DFF()

# compute the next state
ff( TFF.I ^ ff.O )

m.wire(ff.O, TFF.O)
m.wire(TFF.CLK, ff.CLK)

m.EndDefine()


icestick = IceStick()
icestick.Clock.on()
for i in range(len(icestick.J1)):
    icestick.J1[i].output().on()
main = icestick.main()
tff = TFF()
counter = Counter(2)
m.wire(counter.O[1], tff.I)
m.wire(tff.O, main.J1[0])
m.wire(m.bits(0, 7), main.J1[1:])

m.compile("build/magma_tff", main)

compiling XOr2
compiling TFF
compiling FullAdder
compiling Add2Cout
compiling Register2
compiling Counter2
compiling main
Warning (verilog): input CLK not connected to an output


In [5]:
%%bash
yosys -Q -p 'synth_ice40 -top main -blif build/magma_tff.blif' build/magma_tff.v
arachne-pnr -d 1k -o build/magma_tff.txt -p build/magma_tff.pcf build/magma_tff.blif
icepack build/magma_tff.txt build/magma_tff.bin
icetime -tmd hx1k build/magma_tff.txt


-- Parsing `build/magma_tff.v' using frontend `verilog' --

1. Executing Verilog-2005 frontend.
Parsing Verilog input from `build/magma_tff.v' to AST representation.
Generating RTLIL representation for module `\XOr2'.
Generating RTLIL representation for module `\TFF'.
Generating RTLIL representation for module `\FullAdder'.
Generating RTLIL representation for module `\Add2Cout'.
Generating RTLIL representation for module `\Register2'.
Generating RTLIL representation for module `\Counter2'.
Generating RTLIL representation for module `\main'.
Successfully finished Verilog frontend.

-- Running command `synth_ice40 -top main -blif build/magma_tff.blif' --

2. Executing SYNTH_ICE40 pass.

2.1. Executing Verilog-2005 frontend.
Parsing Verilog input from `/usr/local/bin/../share/yosys/ice40/cells_sim.v' to AST representation.
Generating RTLIL representation for module `\SB_IO'.
Generating RTLIL representation for module `\SB_GB_IO'.
Generating RTLIL representation for module `\SB_GB'.
Gener

seed: 1
device: 1k
read_chipdb +/share/arachne-pnr/chipdb-1k.bin...
  supported packages: cb121, cb132, cb81, cm121, cm36, cm49, cm81, qn84, swg16tr, tq144, vq100
read_blif build/magma_tff.blif...
prune...
read_pcf build/magma_tff.pcf...
instantiate_io...
pack...

After packing:
IOs          9 / 96
GBs          0 / 8
  GB_IOs     0 / 8
LCs          3 / 1280
  DFF        3
  CARRY      0
  CARRY, DFF 0
  DFF PASS   0
  CARRY PASS 0
BRAMs        0 / 16
WARMBOOTs    0 / 1
PLLs         0 / 1

place_constraints...
promote_globals...
  promoted 0 nets
  0 globals
realize_constants...
  realized 1
place...
  initial wire length = 73
  at iteration #50: temp = 5.77093, wire length = 34
  at iteration #100: temp = 1.86709, wire length = 27
  at iteration #150: temp = 0.0756508, wire length = 21
  final wire length = 21

After placement:
PIOs       5 / 96
PLBs       2 / 160
BRAMs      0 / 16

  place time 0.00s
route...
  pass 1, 0 shared.

After routing:
span_4     7 / 6944
span_12    1 / 1440
